In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
import joblib
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from xgboost.sklearn import XGBClassifier
import shap 
import matplotlib.pyplot as plt
import warnings

In [ ]:
# Load Dataset
base_dir = r"C:\Users\ChiappiniLab.Chiappini_Lab\Desktop\Sam\20241129_solid_nN_1.3_2.4_mdck_siRNA_tnsfn_chlr\20241129_solid_nN_2.4_mdck_siRNA_dataset"
dataset = base_dir + r"\solid_2.4_siRNA_cell_level.csv"
df = pd.read_csv(dataset)

In [ ]:
# Define morphological and intensity features
cell_morph_features = [
    'area', 'perimeter', 'major_axis_length', 'minor_axis_length', 
    'eccentricity', 'circularity', 'solidity', 'orientation'
]

nuclear_morph_features = ['nuclear_area', 'nuclear_perimeter', 'nuclear_major_axis_length', 'nuclear_minor_axis_length', 
    'nuclear_eccentricity', 'nuclear_circularity', 'nuclear_solidity', 'nuclear_orientation'
]

In [ ]:
channel_feature_suffixes = [ 
    'intensity_p10', 'intensity_p25', 
    'intensity_p50', 'intensity_p75', 
    'intensity_p90'
] 

protein_channels = ['actin', 'caveolin', 'clathrin_hc', 'nuclei']

feature_list = cell_morph_features + nuclear_morph_features + [f"{ch}_{suffix}" for ch in protein_channels for suffix in channel_feature_suffixes]

print(feature_list)

In [ ]:
# Filter cells within 2nd and 98th percentiles and remove negative delivery
cell_area = cell_area_min, cell_area_max = 234, 1927
nuclear_area = nuclei_area_min, nuclei_area_max = 46, 315

# Keep only cells in area range
df_cell_filtered = df[
    (df['area'] >= cell_area_min) &
    (df['area'] <= cell_area_max) 
].copy()

# Keep only nuclei in area range 
nuclei_threshold = (
    (df_cell_filtered['nuclear_area'] >= nuclei_area_min) &
    (df_cell_filtered['nuclear_area'] <= nuclei_area_max)
)

# For rows where the nuclei are out of range, set nucleus area to NaN
nuclear_cols = ['nuclear_area', 'nuclear_perimeter', 'nuclear_major_axis_length', 'nuclear_minor_axis_length', 
    'nuclear_eccentricity', 'nuclear_circularity', 'nuclear_solidity', 'nuclear_orientation', 
    'nuclear_intensity_p10', 'nuclear_intensity_p25', 'nuclear_intensity_p50', 
    'nuclear_intensity_p75', 'nuclear_intensity_p90', ...]

df_cell_filtered.loc[~nuclei_threshold, nuclear_cols] = np.nan

In [ ]:
# -------------------------------
#   Define binary target 
# -------------------------------

# Define target
df_cell_filtered['siRNA_category'] = np.where(
    df_cell_filtered['siRNA_intensity_mean'] > 250,
    1,
    0
)

print("Unique siRNA_category values:", df_cell_filtered['siRNA_category'].unique())
print("Value counts:\n", df_cell_filtered['siRNA_category'].value_counts())

# -------------------------------
#   Define X, y, and images from the *same* df_cell_filtered
# -------------------------------

X = df_cell_filtered[feature_list].copy()
y = df_cell_filtered['siRNA_category'].values
images = df_cell_filtered['image_id'].values

# Encode if needed (binary is not strictly necessary to encode, but okay):
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# -------------------------------
#   Outer Cross-Validation Setup
# -------------------------------
outer_cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
for fold_number, (train_idx, test_idx) in enumerate(outer_cv.split(X, y_encoded, groups=images), start=1):
    y_test_fold = y_encoded[test_idx]
    print(f"Fold {fold_number}: n_test={len(y_test_fold)} -> Class distribution: {np.bincount(y_test_fold)}")

# TRAIN metrics
fold_accuracies_train = []   
fold_precisions_train = []   
fold_recalls_train = []      
fold_f1s_train = []          
fold_aucs_train = []  

# TEST metrics
fold_accuracies_test = []
fold_precisions_test = []
fold_recalls_test = []
fold_f1s_test = []
fold_aucs_test = []


# For aggregated ROC, we store all test labels and probabilities
all_test_true = []
all_test_proba = []

# Track the best overall model (by test accuracy)
best_overall_model = None
best_overall_accuracy = 0.0
best_scaler = None

# -------------------------------
#   Cross-Validation and Hyperparameter Tuning
# -------------------------------
fold_number = 0
for train_idx, test_idx in outer_cv.split(X, y_encoded, groups=images):
    fold_number += 1
    print(f"\n=== Outer Fold {fold_number} ===")
    print(f"Fold {fold_number}: n_test={len(y_test_fold)} -> Class distribution: "
    f"{np.bincount(y_test_fold)} (0s, 1s)")

    # Split data
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define the Optuna objective
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'use_label_encoder': False,  # In newer xgboost versions, this is recommended
            'eval_metric': 'logloss'
        }
        
        model = xgb.XGBClassifier(random_state=42, **params)
        
        # Inner CV for hyperparameter selection
        inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        inner_scores = []
        for inner_train_idx, inner_valid_idx in inner_cv.split(X_train_scaled, y_train):
            X_inner_train = X_train_scaled[inner_train_idx]
            y_inner_train = y_train[inner_train_idx]
            X_inner_valid = X_train_scaled[inner_valid_idx]
            y_inner_valid = y_train[inner_valid_idx]

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.fit(X_inner_train, y_inner_train)
            
            y_pred_inner = model.predict(X_inner_valid)
            score = accuracy_score(y_inner_valid, y_pred_inner)
            inner_scores.append(score)
        
        return np.mean(inner_scores)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)
    best_params = study.best_params
    print("Best parameters:", best_params)

    # Train the final model on this fold using the best parameters
    best_model = xgb.XGBClassifier(
        random_state=42, 
        use_label_encoder=False,
        eval_metric='logloss',
        **best_params
    )
    best_model.fit(X_train_scaled, y_train)

    # ---------------------------
    # Evaluate Training Set (NEW)
    # ---------------------------
    y_train_pred = best_model.predict(X_train_scaled)
    y_train_prob = best_model.predict_proba(X_train_scaled)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred, zero_division=0)
    train_recall = recall_score(y_train, y_train_pred, zero_division=0)
    train_f1 = f1_score(y_train, y_train_pred, zero_division=0)

    # ROC for training (positive class=1)
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_prob[:, 1], pos_label=1)
    train_roc_auc = auc(fpr_train, tpr_train)

    fold_accuracies_train.append(train_accuracy)   
    fold_precisions_train.append(train_precision)  
    fold_recalls_train.append(train_recall)        
    fold_f1s_train.append(train_f1)                
    fold_aucs_train.append(train_roc_auc)          

    # ---------------------------
    # Evaluate Test Set
    # ---------------------------
    y_test_pred = best_model.predict(X_test_scaled)
    y_test_prob = best_model.predict_proba(X_test_scaled)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, zero_division=0)
    test_recall = recall_score(y_test, y_test_pred, zero_division=0)
    test_f1 = f1_score(y_test, y_test_pred, zero_division=0)

    fpr_test, tpr_test, _ = roc_curve(y_test, y_test_prob[:, 1], pos_label=1)
    test_roc_auc = auc(fpr_test, tpr_test)

    # Store test metrics
    fold_accuracies_test.append(test_accuracy)
    fold_precisions_test.append(test_precision)
    fold_recalls_test.append(test_recall)
    fold_f1s_test.append(test_f1)
    fold_aucs_test.append(test_roc_auc)

    # Print immediate fold results
    print(f"Fold {fold_number} Training Accuracy: {train_accuracy:.4f}")
    print(f"Fold {fold_number} Test Accuracy:     {test_accuracy:.4f}")
    print(f"Fold {fold_number} Precision (Test):  {test_precision:.4f}")
    print(f"Fold {fold_number} Recall (Test):     {test_recall:.4f}")
    print(f"Fold {fold_number} F1-score (Test):   {test_f1:.4f}")
    print(f"Fold {fold_number} ROC-AUC (Test):    {test_roc_auc:.4f}")

    # Accumulate for aggregated ROC (test only)
    all_test_true.extend(y_test)
    all_test_proba.extend(y_test_prob)

    # Track best overall model based on test accuracy
    if test_accuracy > best_overall_accuracy:
        best_overall_accuracy = test_accuracy
        best_overall_model = best_model
        best_scaler = scaler

# -------------------------------
#   Aggregate & Print Results
# -------------------------------
print("\n=== Cross-Validation Results (TRAIN) ===")
print(f"Mean Train Accuracy:  {np.mean(fold_accuracies_train):.4f} ± {np.std(fold_accuracies_train):.4f}")
print(f"Mean Train Precision: {np.mean(fold_precisions_train):.4f} ± {np.std(fold_precisions_train):.4f}")
print(f"Mean Train Recall:    {np.mean(fold_recalls_train):.4f} ± {np.std(fold_recalls_train):.4f}")
print(f"Mean Train F1-score:  {np.mean(fold_f1s_train):.4f} ± {np.std(fold_f1s_train):.4f}")
print(f"Mean Train ROC-AUC:   {np.mean(fold_aucs_train):.4f} ± {np.std(fold_aucs_train):.4f}")

print("\n=== Cross-Validation Results (TEST) ===")
print(f"Mean Test Accuracy:  {np.mean(fold_accuracies_test):.4f} ± {np.std(fold_accuracies_test):.4f}")
print(f"Mean Test Precision: {np.mean(fold_precisions_test):.4f} ± {np.std(fold_precisions_test):.4f}")
print(f"Mean Test Recall:    {np.mean(fold_recalls_test):.4f} ± {np.std(fold_recalls_test):.4f}")
print(f"Mean Test F1-score:  {np.mean(fold_f1s_test):.4f} ± {np.std(fold_f1s_test):.4f}")
print(f"Mean Test ROC-AUC:   {np.mean(fold_aucs_test):.4f} ± {np.std(fold_aucs_test):.4f}")

print(f"\nBest overall test accuracy across folds: {best_overall_accuracy:.4f}")

# -------------------------------
#   Save the Best Model
# -------------------------------
joblib.dump((best_overall_model, best_scaler), "best_xgb_model.pkl")
print("Best model saved to 'best_xgb_model.pkl'.")

In [ ]:
"====== Aggregated ROC-AUC curve ====== " 

# "all_test_true" is (N,) with 0/1
# "all_test_proba" is (N,2)

all_test_true = np.array(all_test_true)
all_test_proba = np.array(all_test_proba)

# ROC for positive class (1)
fpr, tpr, _ = roc_curve(all_test_true, all_test_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label=f"Class 1 (AUC = {roc_auc:.2f})")
plt.plot([0,1], [0,1], "k--")  # Diagonal chance line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Class 1)")
plt.legend(loc="lower right")
plt.show()

In [ ]:
"====== Shapley results of best performing fold ====== " 

# 1) Load your best model and scaler
final_model, final_scaler = joblib.load("best_xgb_model.pkl")
print("Loaded best overall model and scaler from 'best_xgb_model.pkl'.")

# 2) Assume you have a test set (X_test, y_test) or a chosen subset
#    For illustration, let's say X_test is already defined:
# X_test = ...  # your DataFrame of test features

# 3) Create a matrix of test values by applying the saved scaler
test_matrix = final_scaler.transform(X_test)

# 4) Create a SHAP explainer for the loaded model
explainer = shap.TreeExplainer(final_model)

# 5) Compute SHAP values for the test matrix
shap_values = explainer.shap_values(test_matrix) 
print("Raw SHAP values shape/type:", type(shap_values), 
      np.array(shap_values).shape if not isinstance(shap_values, list) else "list of arrays")

# ----- Handle Multi-class or Extra Column if Needed -----
# If shap_values is a list or has shape (n_classes, n_samples, n_features),
# select a single class index, e.g. shap_values[0].
if isinstance(shap_values, list):
    shap_values = shap_values[0]
elif shap_values.ndim == 3:
    # Select the SHAP values for a specific class (e.g., class 0) across all samples
    shap_values = shap_values[:, :, 1]

# If there's an extra column for the base value, remove it
if shap_values.shape[1] == test_matrix.shape[1] + 1:
    shap_values = shap_values[:, :-2]

# Double-check the shape matches (n_samples, n_features)
assert shap_values.shape[0] == test_matrix.shape[0], "Row mismatch!"
assert shap_values.shape[1] == test_matrix.shape[1], "Column mismatch!"

# 6) Plot the SHAP beeswarm summary
#    Optionally convert your test_matrix back to a DataFrame with column names
feature_names = X_test.columns.tolist()  # or however you store your feature list
shap.summary_plot(shap_values, test_matrix, feature_names=feature_names)
                                                                                                                                                                                                                                                                                                                                             
# Optionally show the plot inline if in a notebook
plt.show()